In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

import seaborn as sns

import sys
sys.path.append('../code')
import sparseRRR

In [2]:
def preprocess(data, slice_idx, loc_features_idx):
    X = data['Firing rate']
    X = X[slice_idx,:] # select the time slice
    X_mean = np.mean(X, axis=0)
    X_std = np.std(X, axis=0)
    X = X - X_mean
    X = X / X_std

    Y = data['Locomotion']
    Y = Y[slice_idx,:] # select the time slice
    Y = Y[:,loc_features_idx] # select the locomotion features
    Y_mean = np.mean(Y, axis=0)
    Y_std = np.std(Y, axis=0)
    Y = Y - Y_mean
    Y = Y / Y_std
    
    return X,Y, X_mean, X_std, Y_mean, Y_std 

In [3]:
data = pickle.load(open('../data/purkinje_extended_nonlinear.pickle', 'rb'))
locomotion_names = data['locomotion_names']
cell_names = data['cell_names']

Not all timepoints, not all locomotion features:

In [4]:
slice = np.linspace(0, 10000, 10000, dtype=int)
selected_features = [5,6,7,8,9,10,11,12,-8,-7,-6,-5,-4,-3,-2,-1] # locomotion features to use
np.array(locomotion_names)[selected_features]

array(['X-p FR', 'X-p HR', 'X-p FL', 'X-p HL', 'X-s FR', 'X-s HR',
       'X-s FL', 'X-s HL', 'Z-p FR', 'Z-p HR', 'Z-p FL', 'Z-p HL',
       'Z-s FR', 'Z-s HR', 'Z-s FL', 'Z-s HL'], dtype='<U11')

In [5]:
alphas = [0.5, 0.55, 0.67, 0.72, 0.78, 0.79, 0.79, 0.8, 0.8, 0.81]#, 0.9]#, 0.93, 0.98, 1.03, 1.08, 1.13, 1.18, 1.23, 1.28, 1.33]
neurons_selected = []
X_tr,Y_tr, X_tr_mean, X_tr_std, Y_tr_mean, Y_tr_std = preprocess(data, slice, selected_features)
for r in range(1,11):
    print('Rank:', r)
    w,v = sparseRRR.relaxed_elastic_rrr(X_tr, Y_tr, rank=r, alpha=alphas[r-1], l1_ratio=1)
    print('{} neurons selected:'.format(np.sum(w[:,0]!=0)))
    print(cell_names[w[:,0]!=0])
    neurons_selected.append(np.array(cell_names[w[:,0]!=0]))

Rank: 1
10 neurons selected:
[b'MC4017_S2' b'MC5003_S20' b'MC5003_S25' b'MC5003_S30' b'MC5003_S5'
 b'MC5005_S2' b'MC5005_S6' b'MC5006_S10' b'MC5006_S3' b'MC6001_S7']
Rank: 2
10 neurons selected:
[b'MC5003_S1' b'MC5003_S20' b'MC5003_S28' b'MC5003_S30' b'MC5003_S8'
 b'MC5005_S12' b'MC5005_S2' b'MC5005_S6' b'MC5006_S3' b'MC6002_S2']
Rank: 3
10 neurons selected:
[b'MC4017_S7' b'MC5003_S1' b'MC5003_S28' b'MC5003_S30' b'MC5003_S7'
 b'MC5003_S8' b'MC5005_S12' b'MC5006_S10' b'MC5006_S3' b'MC6002_S2']
Rank: 4
10 neurons selected:
[b'MC4017_S7' b'MC5003_S1' b'MC5003_S28' b'MC5003_S30' b'MC5003_S7'
 b'MC5003_S8' b'MC5005_S12' b'MC5006_S10' b'MC5006_S3' b'MC6002_S2']
Rank: 5
10 neurons selected:
[b'MC3810_S1' b'MC4017_S7' b'MC5003_S1' b'MC5003_S28' b'MC5003_S30'
 b'MC5003_S7' b'MC5003_S8' b'MC5005_S12' b'MC5006_S10' b'MC6002_S2']
Rank: 6
10 neurons selected:
[b'MC3810_S1' b'MC4017_S7' b'MC5003_S1' b'MC5003_S28' b'MC5003_S30'
 b'MC5003_S7' b'MC5003_S8' b'MC5005_S12' b'MC5006_S10' b'MC6002_S2']
Rank

In [6]:
for i in range(9):
    print(np.intersect1d(neurons_selected[i], neurons_selected[i+1]))

[b'MC5003_S20' b'MC5003_S30' b'MC5005_S2' b'MC5005_S6' b'MC5006_S3']
[b'MC5003_S1' b'MC5003_S28' b'MC5003_S30' b'MC5003_S8' b'MC5005_S12'
 b'MC5006_S3' b'MC6002_S2']
[b'MC4017_S7' b'MC5003_S1' b'MC5003_S28' b'MC5003_S30' b'MC5003_S7'
 b'MC5003_S8' b'MC5005_S12' b'MC5006_S10' b'MC5006_S3' b'MC6002_S2']
[b'MC4017_S7' b'MC5003_S1' b'MC5003_S28' b'MC5003_S30' b'MC5003_S7'
 b'MC5003_S8' b'MC5005_S12' b'MC5006_S10' b'MC6002_S2']
[b'MC3810_S1' b'MC4017_S7' b'MC5003_S1' b'MC5003_S28' b'MC5003_S30'
 b'MC5003_S7' b'MC5003_S8' b'MC5005_S12' b'MC5006_S10' b'MC6002_S2']
[b'MC3810_S1' b'MC4017_S7' b'MC5003_S1' b'MC5003_S28' b'MC5003_S30'
 b'MC5003_S7' b'MC5003_S8' b'MC5005_S12' b'MC5006_S10' b'MC6002_S2']
[b'MC3810_S1' b'MC4017_S7' b'MC5003_S1' b'MC5003_S28' b'MC5003_S30'
 b'MC5003_S7' b'MC5003_S8' b'MC5005_S12' b'MC5006_S10' b'MC6002_S2']
[b'MC3810_S1' b'MC4017_S7' b'MC5003_S1' b'MC5003_S28' b'MC5003_S30'
 b'MC5003_S7' b'MC5003_S8' b'MC5005_S12' b'MC5006_S10' b'MC6002_S2']
[b'MC3810_S1' b'MC4017_S7